<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-03-02 13:40:17--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-03-02 13:40:17--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  26.1MB/s    in 54s     

2024-03-02 13:41:12 (25.5 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
N = len(wv_embeddings.index_to_key)
print(f"Num of words: {N}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`. Используем косинусное расстояние, поскольку word2vec-представления слов обладают таким свойством, что скалярное произведение векторов, соответствующих близким по смыслу словам, велико.

In [5]:
from numpy.linalg import norm
import numpy as np

def cosine_sim(v1, v2):
    # v1, v2 (1 x dim)
    return v1 @ v2.T / (norm(v1) * norm(v2))

In [6]:
sim = []

cat_index = None

for i in range(N):
    word = wv_embeddings.index_to_key[i]
    if word == 'cat':
        cat_index = i

    sim.append((cosine_sim(wv_embeddings['dog'], wv_embeddings[word]), word))

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [7]:
cat_sim = sim[cat_index][0]
pos = 0
for i in range(N):
    if i != cat_index and sim[i][0] > cat_sim:
        pos += 1

print(f"Cat pos: {pos}")

Cat pos: 26


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [3]:
from nltk.tokenize import WordPunctTokenizer

def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """

    tokens = tokenizer.tokenize(question)
    wv_embeddings = []

    for token in tokens:
        if token in embeddings:
            wv_embeddings.append(embeddings[token][:dim])

    return np.mean(wv_embeddings, axis=0) if len(wv_embeddings) else np.zeros(dim)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [9]:
sent = 'I love neural networks'
word_tokenizer = WordPunctTokenizer()
sent_embedding = question_to_vec(sent, wv_embeddings, word_tokenizer)
print("Third component: ", round(sent_embedding[2], 2))

Third component:  -1.29


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Hits максимизируется при выполнении условия каждого из индикаторов, т.е. если ранг дубликата не превосходит 47. Второй слагаемое (без минуса) не меньше 0 и равно нулю, если условие ни одного из индикаторов не выполнено, т.е. ранг дубликата больше 1. Тогда максимум выражения достигается, например, при ранге 2 каждого из дубликатов и он равен 1.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

$\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2(10)}$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [4]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    dup_ranks = np.array(dup_ranks)
    return np.mean(dup_ranks <= k)

In [5]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dup_ranks = np.array(dup_ranks)
    return np.mean((dup_ranks <= k) / np.log2(1 + dup_ranks))

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [6]:
import pandas as pd

In [69]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [14]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!unzip -u "/content/drive/My Drive/stackoverflow_similar_questions.zip"

Archive:  /content/drive/My Drive/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [9]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))

    return data

Нам понадобится только файл validation.

In [10]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [11]:
len(validation_data)

3760

Размер нескольких первых строк

In [24]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [13]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200, logging=False):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_embedding = question_to_vec(question, embeddings, tokenizer, dim)
    candidate_embeddings = np.array([question_to_vec(candidate, embeddings, tokenizer, dim) for candidate in candidates])
    sim = cosine_similarity(question_embedding[np.newaxis, :], np.array(candidate_embeddings))[0]

    idx = np.argsort(sim, kind='stable')[::-1]
    sim = sim[idx]

    result = []
    to_log = []
    for similarity, i in zip(sim, idx):
        result.append((int(i), candidates[int(i)]))
        to_log.append((similarity, int(i)))

    if logging:
        print("Sim: ", to_log)

    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [27]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [28]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, word_tokenizer, logging=True)
        print(ranks)
        print()

Sim:  [(0.56307507, 1), (0.4788628, 0), (0.16188455, 2)]
[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

Sim:  [(0.48369783, 1), (0.45695466, 0), (0.16329032, 2)]
[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [29]:
# должно вывести
# results = [[(1, 'C# create cookie from string and send it'),
#             (0, 'Convert Google results object (pure js) to Python object'),
#             (2, 'How to use jQuery AJAX for an outside domain?')],
#            [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
#             (*, 'select2 not displaying search results'), #скрыт
#             (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

Попробуем использовать другой токенайзер.

In [ ]:
import numpy as np
import spacy
import string
import nltk
from nltk.corpus import stopwords

class MySpacyTokenizer:
    def __init__(self):
        self.word_tokenizer = WordPunctTokenizer()
        spacy.require_gpu()
        self.nlp = spacy.load('en_core_web_sm')
        nltk.download('stopwords')
        self.stopWords = set(stopwords.words('english'))

    def tokenize(self, text):
        text = text.lower()
        tokens = self.word_tokenizer.tokenize(text)

        new_text = []
        for word in tokens:
            if word in self.stopWords:
                continue
            new_text.append(word)

        tokens = new_text

        spacy_results = self.nlp(' '.join(tokens))
        tokens = [token.lemma_ for token in spacy_results]

        return tokens

my_spacy_tokenizer = MySpacyTokenizer()

In [31]:
for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates(question, q_candidates, wv_embeddings, my_spacy_tokenizer)
    print(ranks)
    print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Результаты второго эксперимента изменились с использованием нового токенайзера.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

Взглянем на первые 10 кандидатов третьего примера из валидационного датасета:

In [ ]:
validation_data[2][:11]

['jQuery: Show a div2 when mousenter over div1 is over',
 'when hover on div1 depenting on if it is on div2 or not it should act differently',
 'How to run selenium in google app engine/cloud?',
 'Python Comparing two lists of strings for similarities',
 'Hazelcast creates 3 nodes/members when configured as Hibernate L2 cache',
 'JSON-LD framing single object arrays',
 'Is there a way to print to the console in an Android app?',
 'Run server-side code on html button press in Express?',
 'Sorting an array by alphabetical order before it is selected by UIPickerView',
 'Node - Prevent child process from going in to parent directory',
 'No Empty Trash scheduled task in Sonatype Nexus OSS 3']

In [15]:
from tqdm.notebook import tqdm

### 1. Mode: удаление пунктуации, лемматизация слов при помощи nltk, удаление стоп-слов

In [75]:
import numpy as np
import string
import nltk
from nltk.corpus import stopwords

class MyNLTKTokenizer:
    def __init__(self):
        self.word_tokenizer = WordPunctTokenizer()
        self.wnl = nltk.WordNetLemmatizer()
        nltk.download('stopwords')
        nltk.download('wordnet')
        self.stopWords = set(stopwords.words('english'))

    def tokenize(self, text):
        for c in string.punctuation:
            text = text.replace(c, '')

        text = text.lower()
        tokens = self.word_tokenizer.tokenize(text)

        new_text = []
        for word in tokens:
            if word in self.stopWords or len(word) < 2:
                continue
            new_text.append(word)

        tokens = new_text
        tokens = [self.wnl.lemmatize(word) for word in tokens]

        return tokens

my_nltk_tokenizer = MyNLTKTokenizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [85]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, my_nltk_tokenizer)
    pos = list(np.array(ranks)[:, 0].astype(int)).index(0)
    wv_ranking.append(pos + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [86]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.371 | Hits@   1: 0.371
DCG@   5: 0.471 | Hits@   5: 0.561
DCG@  10: 0.492 | Hits@  10: 0.629
DCG@ 100: 0.535 | Hits@ 100: 0.841
DCG@ 500: 0.551 | Hits@ 500: 0.958
DCG@1000: 0.555 | Hits@1000: 1.000


### 2. Mode: удаление пунктуации, стемминг с использованием PorterStemmer из nltk, удаление стоп-слов

In [92]:
from nltk.stem import PorterStemmer

class MyStemTokenizer:
    def __init__(self):
        self.word_tokenizer = WordPunctTokenizer()
        self.ps = PorterStemmer()
        nltk.download('stopwords')
        self.stopWords = set(stopwords.words('english'))

    def tokenize(self, text):
        for c in string.punctuation:
            text = text.replace(c, '')

        text = text.lower()
        tokens = self.word_tokenizer.tokenize(text)

        new_text = []
        for word in tokens:
            if word in self.stopWords or len(word) < 2:
                continue
            new_text.append(word)

        tokens = new_text
        tokens = [self.ps.stem(word) for word in tokens]

        return tokens

my_stem_tokenizer = MyStemTokenizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [93]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, my_stem_tokenizer)
    pos = list(np.array(ranks)[:, 0].astype(int)).index(0)
    wv_ranking.append(pos + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [94]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.297 | Hits@   1: 0.297
DCG@   5: 0.369 | Hits@   5: 0.434
DCG@  10: 0.394 | Hits@  10: 0.510
DCG@ 100: 0.447 | Hits@ 100: 0.770
DCG@ 500: 0.467 | Hits@ 500: 0.931
DCG@1000: 0.475 | Hits@1000: 1.000


### 3. Mode: Стемминг с использованием PorterStemmer из nltk

In [95]:
from nltk.stem import PorterStemmer

class MyStemTokenizer2:
    def __init__(self):
        self.word_tokenizer = WordPunctTokenizer()
        self.ps = PorterStemmer()

    def tokenize(self, text):
        text = text.lower()
        tokens = self.word_tokenizer.tokenize(text)
        tokens = [self.ps.stem(word) for word in tokens]

        return tokens

my_stem_tokenizer2 = MyStemTokenizer2()

In [96]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, my_stem_tokenizer2)
    pos = list(np.array(ranks)[:, 0].astype(int)).index(0)
    wv_ranking.append(pos + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [97]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.317 | Hits@   1: 0.317
DCG@   5: 0.395 | Hits@   5: 0.465
DCG@  10: 0.419 | Hits@  10: 0.538
DCG@ 100: 0.471 | Hits@ 100: 0.797
DCG@ 500: 0.490 | Hits@ 500: 0.949
DCG@1000: 0.496 | Hits@1000: 1.000


### 4. Mode: лемматизация слов при помощи nltk

In [22]:
import numpy as np
import string
import nltk
from nltk.corpus import stopwords

class MyNLTKTokenizer2:
    def __init__(self):
        self.word_tokenizer = WordPunctTokenizer()
        nltk.download('wordnet')
        self.wnl = nltk.WordNetLemmatizer()

    def tokenize(self, text):
        text = text.lower()
        tokens = self.word_tokenizer.tokenize(text)
        tokens = [self.wnl.lemmatize(word) for word in tokens]

        return tokens

my_nltk_tokenizer2 = MyNLTKTokenizer2()

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [23]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, my_nltk_tokenizer2)
    pos = list(np.array(ranks)[:, 0].astype(int)).index(0)
    wv_ranking.append(pos + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [24]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.386 | Hits@   1: 0.386
DCG@   5: 0.484 | Hits@   5: 0.569
DCG@  10: 0.505 | Hits@  10: 0.631
DCG@ 100: 0.553 | Hits@ 100: 0.869
DCG@ 500: 0.566 | Hits@ 500: 0.966
DCG@1000: 0.570 | Hits@1000: 1.000


Таким образом, мы получаем лучшие значения метрик при использовании последнего токенайзера.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [72]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

Обучать модель будем с использованием токенайзера MyNLTKTokenizer

In [77]:
train_data = [my_nltk_tokenizer.tokenize(line[0]) + my_nltk_tokenizer.tokenize(line[1]) for line in train_data]

In [80]:
sum = 0
for line in train_data:
    if len(line) > sum:
        sum = len(line)

sum

37

В качестве размера окна возьмём половину от длины самого длинного предложения тренировочной выборки. Таким образом, мы даём модели возможность выучить "взаимосвязи" слов в контексте похожих предложений.

In [81]:
from gensim.models import Word2Vec

window_width = 20
embeddings_trained = Word2Vec(train_data,
                vector_size=256,
                min_count=50,
                window=window_width).wv

Провалидируемся:

In [82]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, my_nltk_tokenizer, dim=256)
    pos = list(np.array(ranks)[:, 0].astype(int)).index(0)
    wv_ranking.append(pos + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [83]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.402 | Hits@   1: 0.402
DCG@   5: 0.506 | Hits@   5: 0.597
DCG@  10: 0.527 | Hits@  10: 0.661
DCG@ 100: 0.570 | Hits@ 100: 0.866
DCG@ 500: 0.582 | Hits@ 500: 0.959
DCG@1000: 0.586 | Hits@1000: 1.000


Результат работы с использованием первого токенайзера улучшился!

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


При решении задачи ранжирования на предобученных эмбеддингах наилучшее значение метрик удалось достичь с использованием токенайзера, который включал в себя только лемматизацию. Лемматизация (нормализация слов) действительно даёт прирост в качестве.

При решении задачи ранжирования на обученных нами эмбеддингах лучше себя показал токенайзер, который включает в себя удаление пунктуационных символов, удаление стоп-слов, а также лемматизацию. Полученные результаты можно объяснить тем, что предложения становятся длиннее и для выделения важной информации из контекста удаление стоп-слов и пунктуации становится более необходимым.

Если сравнивать предобученные эмбеддинги с обученными нами, то значения метрик при использовании первых $-$ выше. Наверное, это можно объяснить бОльшим корпусом предложений со StackOverflow. Наверное, ещё большую роль сыграл выбор автором лучшей комбинации параметров для обучения этих эмбеддингов. Возможно, перебрав ещё некоторые варианты обучения наших собственных эмбеддингов, мы бы побили скор, полученный на предобученных эмбеддингах.

В качестве другого подхода решения задачи ранжирования я бы, как было отмечено в ноутбуке выше, добавил n-ое число слоёв нейросети для ранжирования эмбеддингов предложений.